<a href="https://colab.research.google.com/github/rohandawar/pyspark/blob/main/OLS_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I am trying to do the correlation analysis using pyspark, also the initial cells install the pyspark and handles the data imports.

Firstly, Since Pyspark is not a default library in google colab, we need to install the same in runtime, post which we can start performing the analysis

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=05d7abb97536c70d25940fa85ade1d24194af00b746a762f4e9981dee1667056
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [29]:
# Import Libs

# Pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# google colab
from google.colab import drive

In [5]:
# create a spark session
spark = SparkSession.builder.appName('LinearRegression').getOrCreate()
# spark = SparkSession.builder.appName('CorrelationAnalysis').getOrCreate()

In [6]:
# Mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Read the data
path = '/content/drive/MyDrive/DataSets_Pyspark_GoogleColab_Primer/Boston.csv' # Path for the items in the google drive
df1 = spark.read.csv(path, inferSchema=True, header=True).drop('_c0')
df1.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [25]:
# Convert the features into a single Vector

# Creating a list of columns
Vector_input = [x for x in df1.columns if x !='medv']

vec_assembler = VectorAssembler(inputCols=Vector_input, outputCol = 'features' )

# fit the VectorAssemebler
finaldf = vec_assembler.transform(df1)
finaldf.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|            features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows



In [27]:
# Perform the train & test SPlit
train_df, test_df = finaldf.randomSplit([0.70,0.30], seed=42)

In [28]:
# instiate the Regressor
Regressor = LinearRegression(labelCol='medv', featuresCol='features')

# Train the model
lr_model = Regressor.fit(train_df)

# make Predictions
pred_df = lr_model.transform(test_df)
pred_df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|            features|        prediction|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0|0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|27.415548111956653|
|0.01381|80.0| 0.46|   0|0.422|7.875|32.0|5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...| 41.02054163950049|
|0.01439|60.0| 2.93|   0|0.401|6.604|18.8|6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...|31.457706876789025|
|0.01501|80.0| 2.01|   0|0.435|6.635|29.7| 8.344|  4|280|   17.0|390.94| 5.99|24.5|[0.01501,80.0,2.0...|27.800872027630003|
|0.01778|95.0| 1.47|   0|0.403|7.135|13.9|7.6534|  3|402|   17.0| 384.3| 4.45|32.9|[0.01778,95.0,1.4...| 30.78816002482059|
+-------

In [32]:
# Evalution
regression_evaluator = RegressionEvaluator(labelCol='medv', predictionCol='prediction', metricName='rmse')
rmse = regression_evaluator.evaluate(pred_df)
r2 = regression_evaluator.setMetricName('r2').evaluate(pred_df)
print(f"RMSE: {rmse}")
print(f"r2: {round(r2*100,2)}%")

RMSE: 5.795278736539351
r2: 65.06%
